# \\Add the logo here\\

**Scale your pandas workflows by changing one line of code**

## Exercise 3: Not Implemented and Looping

**GOAL**: Learn what happens when a function is not yet supported in Modin and what functionality is not possible to accelerate

When functionality has not yet been implemented, we default to pandas

# \\Add figure here\\

We convert a Modin dataframe to pandas to do the operation, then convert it back once it is finished. These operations will have a high overhead due to the communication involved and will take longer than pandas.

When this is happening, a warning will be given to the user to inform them that this operation will take longer than usual. For example, `DataFrame.kurtosis` is not yet implemented. In this case, when a user tries to use it, they will see this warning:

```
UserWarning: `DataFrame.kurtosis` defaulting to pandas implementation.
```

## Concept for exercise: Default to pandas

In this section of the exercise we will see first-hand how the runtime is affected by operations that are not implemented.

In [ ]:
import modin.pandas as pd
import numpy as np
import time

frame_data = np.random.randint(0, 100, size=(2**12, 2**8))
df = pd.DataFrame(frame_data).add_prefix("col")

In [ ]:
start = time.time()

kurtosis = df.kurtosis()

end = time.time()
print("Modin kurtosis took {} seconds.".format(end - start))

In [ ]:
kurtosis

In [ ]:
type(kurtosis)